In [1]:
import numpy as np
import librosa.display
import librosa
from sklearn import model_selection
from pathlib import Path

import pandas as pd
import tqdm
from pathlib import Path


## Load GTZAN data

In [2]:
data, labels, sr, failed = [], [], [], []

for f in tqdm.tqdm(Path('./').glob('genres_original/*/*.wav')):
    
    try:
        x, s = librosa.load(f)
    except Exception as e:   
        failed.append(f)
        continue
        
    data.append(x[:660000])
    sr.append(s)

    labels.append(str(f).split('/')[1])
    

929it [00:05, 278.64it/s]/var/folders/sb/d9wj7wtj5r71by92btgtpggc0000gq/T/ipykernel_29291/3016074573.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  x, s = librosa.load(f)
/Users/sean.fitzgibbon/dev/RP/.venv/lib/python3.8/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Exception ignored in: <function CFObject.__del__ at 0x16a83cf70>
Traceback (most recent call last):
  File "/Users/sean.fitzgibbon/dev/RP/.venv/lib/python3.8/site-packages/audioread/macca.py", line 135, in __del__
    _corefoundation.CFRelease(self._obj)
AttributeError: 'CFURL' object has no attribute '_obj'
Exception ignored in: <function ExtAudioFile.__del__ at 0x15622c790>
Traceback (most recent call last):
  File "/Users/sean.fitzgibbon/dev/RP/.venv/lib/python3.8/site-packages/audioread/macca.py", line 336,

In [4]:
len(data), len(labels), len(sr), len(failed)

(999, 999, 999, 1)

In [5]:
failed

[PosixPath('genres_original/jazz/jazz.00054.wav')]

In [10]:
sr[0]

22050

In [11]:
np.unique(labels)

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype='<U9')

In [12]:
data = np.stack(data, axis=0) 
sr = np.array(sr)
labels = np.array(labels)

data.shape, sr.shape, labels.shape

((999, 660000), (999,), (999,))

In [15]:
data_train, data_held, labels_train, labels_held, sr_train, sr_held = model_selection.train_test_split(
    data, 
    labels,
    sr,
    test_size=0.2,
    stratify=labels,
)

In [16]:
data_train.shape, labels_train.shape, sr_train.shape

((799, 660000), (799,), (799,))

In [17]:
data_held.shape, labels_held.shape, sr_held.shape

((200, 660000), (200,), (200,))

In [18]:
outpath = Path('data_npy/gtzan')
outpath.mkdir(parents=True, exist_ok=True)

np.save(outpath / 'data_train.npy', data_train)
np.save(outpath / 'data_heldout.npy', data_held)
np.save(outpath / 'labels_train.npy', labels_train)
np.save(outpath / 'labels_heldout.npy', labels_held)
np.save(outpath / 'sr_train.npy', sr_train)
np.save(outpath / 'sr_heldout.npy', sr_held)

## Load FMA data

In [16]:
sample_rate = 44100

# load tracks metadata
tracks = pd.read_csv(Path('FMA/tracks_genre.csv')).set_index('track_id')

# create genre mapping for files we have
mapping = []

for f in Path('./FMA').glob('fma_small/*/*.mp3'):
    mapping.append(
        {
            'track_id': int(f.stem),
            'genre': tracks.loc[int(f.stem), 'genre_top'],
            'filename': f,
        }
    )

mapping = pd.DataFrame(mapping)

# sample the filenames stratified on genre (to match GTZAN numbers for testing)

# TODO: remove subsampling once it works

training_size = 1000

filenames, _, genres, _ = model_selection.train_test_split(
    mapping.filename, 
    mapping.genre, 
    train_size=training_size, 
    stratify=mapping.genre
)

# load the data

data, sr, labels, failed = [], [], [], []

for f, genre in tqdm.tqdm(zip(filenames, genres)):

    try:
        y, sr0 = librosa.load(f, sr=sample_rate)
        
        data.append(y[:1321438])
        sr.append(sr0)
        labels.append(genre)
    except:
        failed.append(f)
        continue

876it [00:37, 24.35it/s][src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
1000it [00:42, 23.68it/s]


In [17]:
y.shape, len(data), len(sr), len(labels), len(failed)

((1321967,), 1000, 1000, 1000, 0)

In [18]:
data = np.stack(data, axis=0) 
labels = np.array(labels)
sr = np.array(sr)

In [19]:
data.shape, labels.shape, sr.shape

((1000, 1321438), (1000,), (1000,))

In [24]:
sr[0]

44100

In [20]:
data_train, data_held, labels_train, labels_held, sr_train, sr_held = model_selection.train_test_split(
    data, 
    labels,
    sr,
    test_size=0.2,
    stratify=labels,
)

In [21]:
data_train.shape, labels_train.shape, sr_train.shape

((800, 1321438), (800,), (800,))

In [22]:
data_held.shape, labels_held.shape, sr_held.shape

((200, 1321438), (200,), (200,))

In [23]:
outpath = Path('data_npy/fma_small')
outpath.mkdir(parents=True, exist_ok=True)

np.save(outpath / 'data_train.npy', data_train)
np.save(outpath / 'data_heldout.npy', data_held)
np.save(outpath / 'labels_train.npy', labels_train)
np.save(outpath / 'labels_heldout.npy', labels_held)
np.save(outpath / 'sr_train.npy', sr_train)
np.save(outpath / 'sr_heldout.npy', sr_held)

In [26]:
outpath = Path('data_npy/fma_small_22k')
outpath.mkdir(parents=True, exist_ok=True)

print(data.shape)
data = librosa.resample(data, orig_sr=sr[0], target_sr=22050, axis=1)
print(data.shape)

np.save(outpath / 'data_train.npy', data_train)
np.save(outpath / 'data_heldout.npy', data_held)
np.save(outpath / 'labels_train.npy', labels_train)
np.save(outpath / 'labels_heldout.npy', labels_held)
np.save(outpath / 'sr_train.npy', sr_train)
np.save(outpath / 'sr_heldout.npy', sr_held)

(1000, 1321438)
(1000, 660719)
